We solve the following weak formulation of the linear Darcy problem:


\begin{equation}
    \text{Find} \ \ (\boldsymbol{u},p)\in \boldsymbol{W}\times Q  \ \ \text{s.t.} \ \ \begin{cases}
       \int_{\Omega}\nu\boldsymbol{u}\cdot\boldsymbol{v} - \int_{\Omega} p\,\text{div}\boldsymbol{v} = \int_{\Omega}\boldsymbol{f}\cdot\boldsymbol{v} - \int_{\partial\Omega} p_D\,\boldsymbol{n} \cdot \boldsymbol{v}\quad &\forall \boldsymbol{v}\in \boldsymbol{W} \\\int_{\Omega} q\,\text{div}\boldsymbol{u} \,= 0 \quad &\forall q \in Q  
    \end{cases}
\end{equation}


we then discretize it choosing the spaces:


\begin{equation*}
    \mathbb{R}\mathbb{T}^{\,r}(\mathcal{K})\, = \mathbb{P}^{\,r}(\mathcal{K})^2 \bigoplus x\mathbb{P}^{\,r}(\mathcal{K})
\end{equation*}
\begin{equation*}
    \boldsymbol{W}_h = \{\boldsymbol{v}_h\in \boldsymbol{H}(\text{div};\mathcal{T}_h)\,, \;\;\boldsymbol{v}_h|_{\mathcal{K}} \in \mathbb{R}\mathbb{T}^{\,r}(\mathcal{K})\,,\;\;\forall\mathcal{K}\in\mathcal{T}_h\}\,,
\end{equation*}
\begin{equation*}
    Q_h = \mathbb{P}^{r-1}({\mathcal{T}_h})
\end{equation*}


Hence, by gathering the terms in the bilinear forms and in the functional we solve:



\begin{equation}
    \text{Find}\quad(\boldsymbol{u}_h,p_h)\in\boldsymbol{W_h}\times Q_h\quad st\quad \begin{cases}
        a_h(\boldsymbol{u}_h,\boldsymbol{v}_h) + b_h(\boldsymbol{v}_h,p_h) = F_h(\boldsymbol{v}_h)\quad&\forall\boldsymbol{v}_h\in\boldsymbol{W}_h \\ b_h(\boldsymbol{u}_h,q_h) = 0 &\forall q_h \in Q_h
    \end{cases}
\end{equation}

In [1]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [2]:
from fenics import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
# First Experiment - Polynomial case

nu = Constant(0.1)
f = Constant((0.0, 0.0))
p_exact = Expression('x[0]*x[1]', degree=3)
u_exact = Expression(('-(1/nu)*x[1]', '-(1/nu)*x[0]'), degree=2, nu=nu)

d = 100
degree = 1 #r

In [3]:
# Second experiment - trigonometric case

nu = Constant(1.0)
f = Expression(('nu*pow(sin(x[1]),2)+cos(x[0])*cos(x[1])','nu*pow(cos(x[0]),2)-sin(x[0])*sin(x[1])'), nu=nu, degree=4)
p_exact = Expression('sin(x[0])*cos(x[1])', degree=2)
u_exact = Expression(('pow(sin(x[1]),2)','pow(cos(x[0]),2)'), degree=4)

d = 100
degree = 2

In [4]:
def LinearDarcy(d, degree, nu, f, u_exact, p_exact):

    # 1. mesh generation
    square = Rectangle(Point(0.0, 0.0), Point(1.0, 1.0))
    mesh = generate_mesh(square, d)

    # 2. definition of a MFE stable coupling
    U = FiniteElement('RT', mesh.ufl_cell(), degree)
    P = FiniteElement('DG', mesh.ufl_cell(), degree-1)
    X = FunctionSpace(mesh, U * P)

    # 3. assembling
    u, p = TrialFunctions(X)
    v, q = TestFunctions(X)
    n = FacetNormal(mesh)

    a = ((nu) * dot(u, v) - q * div(u) - p * div(v)) * dx
    L = dot(f, v) * dx - p_exact * dot(v, n) * ds

    # 4. solve
    x = Function(X)
    solve(a == L, x)
    uh, ph = x.split()

    return uh, ph

Now we compute the errors of the discrete solutions

In [ ]:
# Compute the errors
error_L2  = np.empty([2,4])
error_H1  = np.empty([2,4])
error_div = np.empty([1,4])

h = np.empty([1,4])
i = 0

for n in [20, 40, 80, 160]:
  u, p = LinearDarcy(n, degree, nu, f, u_exact, p_exact)
  error_L2[0,i] = errornorm(u_exact,u,'L2')
  error_L2[1,i] = errornorm(p_exact,p,'L2')
  error_H1[0,i] = errornorm(u_exact,u,'H10')
  error_H1[1,i] = errornorm(p_exact,p,'H10')
  error_div[0,i] = errornorm(u_exact,u,'Hdiv0')
  h[0,i] = 1/n
  i = i+1

In [ ]:
# Error rescaling to see better plots
h = h / h[0,0]
error_L2[0,:] = error_L2[0,:] / error_L2[0,0]
error_L2[1,:] = error_L2[1,:] / error_L2[1,0]
error_H1[0,:] = error_H1[0,:] / error_H1[0,0]
error_H1[1,:] = error_H1[1,:] / error_H1[1,0]
error_div[0,:] = error_div[0,:] / error_div[0,0]

In [ ]:
# Plots of the errors
plt.loglog(h[0,:], h[0,:], 'y', label='order 1')
plt.loglog(h[0,:], h[0,:]**2, 'r', label='order 2')
plt.loglog(h[0,:], error_L2[0,:], 'c--o', label='u_L2')
plt.loglog(h[0,:], error_L2[1,:], 'm--o', label='p_L2')

plt.title("Orders of convergence Linear Darcy");
plt.xlabel("h")
plt.ylabel("Errors")
plt.legend(loc=4)

plt.show()

In [ ]:
# Compute the orders of convergence
order_L2 =np.empty([2,3])
order_H1 =np.empty([2,3])
order_div=np.empty([1,3])

for j in [1,2,3]:
  order_L2[0,j-1]=math.log(error_L2[0,j-1]/error_L2[0,j],2)
  order_L2[1,j-1]=math.log(error_L2[1,j-1]/error_L2[1,j],2)
  order_H1[0,j-1]=math.log(error_H1[0,j-1]/error_H1[0,j],2)
  order_H1[1,j-1]=math.log(error_H1[1,j-1]/error_H1[1,j],2)
  order_div[0,j-1]=math.log(error_div[0,j-1]/error_div[0,j],2)

print(order_L2)
print(order_H1)
print(order_div)